<a href="https://colab.research.google.com/github/manishiitg/ML_Experments/blob/master/keras/simple_FF_deep_learning_classifier_with_word2vec_embedding_%2B_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP Simple Text Classification with Keras!

This is a very simple model using word2vec embeddings + TF-IDF

This is just to experiment and see how this performs and as expected the results are low.

Need to look at LSTM or CNN models for better learning.



In [0]:
%matplotlib inline

import pandas as pd
import numpy as np

import spacy 
nlp = spacy.load("en_core_web_sm")

from sklearn.datasets import fetch_20newsgroups

from gensim.models import KeyedVectors

from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;

from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
import matplotlib.pyplot as plt;
from sklearn.model_selection import train_test_split


from sklearn.feature_extraction.text import TfidfVectorizer


from collections import defaultdict


In [0]:
def normalize(comment, lowercase=True, remove_stopwords=True):
    if lowercase:
        comment = comment.lower()
    lines = comment.splitlines()
    lines = [x.strip(' ') for x in lines]
    lines = [x.replace('"', '') for x in lines]
    lines = [x.replace('\\"', '') for x in lines]
    lines = [x.replace(u'\xa0', u'') for x in lines]
    comment = " ".join(lines)
    doc = nlp(comment)

    # for token in doc:
    #   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #     token.shape_, token.is_alpha, token.is_stop)

    words = [token for token in doc if token.is_stop !=
             True and token.is_punct != True]
    # return " ".join(words)
    lemmatized = list()
    for word in words:
        lemma = word.lemma_.strip()
        if lemma:
            lemmatized.append(lemma)
    return lemmatized

In [3]:
import os

def writetofile(dir, filename, data):
    if not os.path.exists(dir):
        os.makedirs(dir)
    f = os.path.join(dir,str(filename))

    with open(f, 'wb') as the_file:
      the_file.write(data)


categories = [ 'comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','comp.windows.x']
news = fetch_20newsgroups(subset="train", categories=categories)


clean_data  = []

# print(news.keys())

# print(news["filenames"][:10])

# print(len(news["data"][:1000]))

# print(news["target_names"][:50])
# print(news["target"][:10])


max_limit = 4000

targets = news["target"][:max_limit]
filenames = news["filenames"][:max_limit]
news = news["data"][:max_limit]

dir = "news_group_cleaned"

print("cleaning data")
for i, row in enumerate(news):
  filename = filenames[i]
  filename = filename[(filename.rfind('/'))+1:]
  if os.path.exists(filename):
    with open(os.path.join(dir, filename), 'r') as content_file:
      data = content_file.read()
      cleaned = data.split(" ")
  else:
    cleaned = normalize(row)
    writetofile(dir, filename, " ".join(cleaned).encode("utf-8"))
    
  
  clean_data.append(cleaned)

print("data cleaned")

# print(clean_data[10])

cleaning data
data cleaned


In [4]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = gensim.models.Word2Vec(clean_data)

#https://stackoverflow.com/questions/45159693/word2vec-models-consist-of-characters-instead-of-words

2019-11-08 05:32:56,344 : INFO : collecting all words and their counts
2019-11-08 05:32:56,345 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-08 05:32:56,463 : INFO : collected 66979 word types from a corpus of 470869 raw words and 2936 sentences
2019-11-08 05:32:56,464 : INFO : Loading a fresh vocabulary
2019-11-08 05:32:56,503 : INFO : effective_min_count=5 retains 7758 unique words (11% of original 66979, drops 59221)
2019-11-08 05:32:56,504 : INFO : effective_min_count=5 leaves 392121 word corpus (83% of original 470869, drops 78748)
2019-11-08 05:32:56,535 : INFO : deleting the raw counts dictionary of 66979 items
2019-11-08 05:32:56,539 : INFO : sample=0.001 downsamples 33 most-common words
2019-11-08 05:32:56,540 : INFO : downsampling leaves estimated 326228 word corpus (83.2% of prior 392121)
2019-11-08 05:32:56,571 : INFO : estimated required memory for 7758 words and 100 dimensions: 10085400 bytes
2019-11-08 05:32:56,572 : INFO : resetting 

In [0]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = 100

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        #https://stackoverflow.com/a/5900634
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

w2v = dict(zip(model.wv.index2word, model.wv.vectors))

tf = TfidfEmbeddingVectorizer(w2v)
tf.fit(clean_data, targets)
embeddings = tf.transform(clean_data)   



In [6]:
print(len(embeddings))
print(len(targets))

X_train, X_test, Y_train, Y_test = train_test_split(embeddings, targets, test_size=0.33, random_state=42)

2936
2936


In [7]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()

model.add(Dense(units=len(X_test), activation='relu', input_dim=100))
model.add(Dense(units=5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()


model.fit(X_train, Y_train,
          epochs=5, batch_size=128, verbose=1, validation_split=.2)


scores = model.evaluate(X_test, Y_test, verbose=1)


print("Accuracy: ", scores[1])
print("Loss: ", scores[0])


Using TensorFlow backend.


2019-11-08 05:33:48,324 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

2019-11-08 05:33:48,335 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

2019-11-08 05:33:48,343 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

2019-11-08 05:33:48,377 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2019-11-08 05:33:48,400 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3622: The name tf.log is deprecated. Please use tf.math.log 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 969)               97869     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 4850      
Total params: 102,719
Trainable params: 102,719
Non-trainable params: 0
_________________________________________________________________


2019-11-08 05:33:48,659 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

2019-11-08 05:33:48,701 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3005: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-08 05:33:48,709 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

2019-11-08 05:33:48,710 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

2019-11-08 05:33:48,744 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use 

Train on 1573 samples, validate on 394 samples
Epoch 1/5


2019-11-08 05:33:48,960 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



1573/1573 [==============================] - 1s 664us/step - loss: 1.4017 - acc: 0.3821 - val_loss: 1.2129 - val_acc: 0.4518
Epoch 2/5
1573/1573 [==============================] - 0s 114us/step - loss: 1.1571 - acc: 0.4660 - val_loss: 1.1336 - val_acc: 0.4822
Epoch 3/5
1573/1573 [==============================] - 0s 121us/step - loss: 1.1061 - acc: 0.5270 - val_loss: 1.1412 - val_acc: 0.4746
Epoch 4/5
1573/1573 [==============================] - 0s 91us/step - loss: 1.0935 - acc: 0.5308 - val_loss: 1.0620 - val_acc: 0.5305
Epoch 5/5
969/969 [==============================] - 0s 39us/step
Accuracy:  0.5356037154163247
Loss:  1.0854043298707534


As we see the score is quite low. simple NN are not suitable for text based neural nets, we will look at CNN or LSTM i.e RNN